In [1]:
import pandas as pd
import numpy as np
import warnings
from gekko import GEKKO
warnings.filterwarnings("ignore")
import os

In [10]:
os.chdir("C:/Users/RobertoRosasGuerrero/Downloads")
NameLibr = 'TV_Prueba2.xlsx'
##############################
########Definimos el número de segmentos que vamos a alinear debe ser entre 2 y 4

#############   IMPORTANTE DEFINIR  ##########################
segmentos = 2  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


 

In [11]:
#Hacemos la lectura de las tablas 
if(segmentos>=2):
    #validation tables, train/test
    tv1 = pd.read_excel(NameLibr, sheet_name=0)
    tv2 = pd.read_excel(NameLibr, sheet_name=1)
    if(segmentos>=3):
        tv3 = pd.read_excel(NameLibr, sheet_name=2)
    else: pass    

    if(segmentos>=4):
        tv4 = pd.read_excel(NameLibr, sheet_name=3)
    else: pass   


###### Generamos la calibratedScore ###############################
###################################################################

##### Damos los parametros para generar el Score Calibrado #####

MinScore = 300
Incremento = 120

######## Damos los parametros para uncalibrated data ###########

minSC = 0    # 0 min y 1,000 max  para dar inicio, ya después lo vamos modificando
maxSC = 1000 

lst_odds = [1,2,4,8,16]
calibred = pd.DataFrame({'SCORE': [str(MinScore+i*Incremento)+"-" for i in range(5)],
                         'ODDS':  lst_odds,
                         'Log(Odds)': list(map(np.log, lst_odds))})
#calibred                         



################################################################################################################
############################### Definimos las funciones ########################################################
################################################################################################################
# Generamos la calibratedScore  gentvc
def aa(MiS,Inc):
    MiS = int(MiS)
    Inc = int(Inc)
    dff = pd.DataFrame({'SCORE': [str(MiS+i*Inc)+"-" for i in range(5)],
                         'ODDS':  lst_odds,
                         'Log(Odds)': list(map(np.log, lst_odds))})
    return dff
#calibred = gentvc(MinScore,Incremento)       
#calibred
#calcular_descripcion_linea
def bb(TV):
    '''Función para calcular las propiedades de la recta a partir de la tabla de validación (como DataFrame).
       Calcula la pendiente y la ordenada al origen de la recta que ajusta a los pares de puntos
       x = score minimo de la banda en la tabla de validacion, y = odds de la tabla de validación.'''
    t_uncalibred = pd.DataFrame({'Score': list(map(lambda x: int(x.split('-')[0]), TV['SCORE'][:-1])),
                                 'Odds':  TV['ODDS'][:-1],
                                 'Log(Odds)': list(map(np.log, TV['ODDS'][:-1]))})
    pendiente, ordenada = tuple(np.polyfit(t_uncalibred['Score'], t_uncalibred['Log(Odds)'],1))
    return pendiente, ordenada    

##calculamos el valor del score de la tabla de validacion CalVal
def cc(Pen,Ord,PenCal,OrdCal,Sco):
    dd = (Pen/(PenCal) * Sco) + ((Ord-(OrdCal))/(PenCal) )
    return dd  

In [12]:
##################################################################################################################################################################
#################################################### Prueba para automatizar el número de tablas de validación ####################################################
###################################################################################################################################################################

#from more_itertools import value_chain
m = GEKKO(remote=False) #remote=False
# Use IPOPT solver (default)
m.options.SOLVER = 3
# Change to parallel linear solver
#m.solver_options = ['linear_solver ma97']

def opti(NumSeg):
    if((NumSeg>1)|(NumSeg<5)):    
        #from more_itertools import value_chain
        m = GEKKO(remote=False) #remote=False
        # Use IPOPT solver (default)
        m.options.SOLVER = 3
        # Change to parallel linear solver
        #m.solver_options = ['linear_solver ma97']

        minSC = 0    # 0 min y 1,000 max  para dar inicio, ya después lo vamos modificando
        maxSC = 1000 

        c1P = bb(tv1)[0]
        c1O = bb(tv1)[1]

        c2P = bb(tv2)[0]
        c2O = bb(tv2)[1]

        if(NumSeg>=3):
            c3P = bb(tv3)[0]
            c3O = bb(tv3)[1]
            if(NumSeg==4):
                    c4P = bb(tv4)[0]
                    c4O = bb(tv4)[1]
            else: pass
        else: pass


        #bb(aa(X1,X2))[0]
        X1 =  m.Var(value = 150, lb = 0, ub = 500)  #MinScore
        X2 =  m.Var(value = 120, lb = 50, ub = 400)  #Incremento
        X3 =  m.Var(bb(aa(str(X1.value),str(X2.value)))[0]) #X3  Pendiente Calibrado  
        X4 =  m.Var(bb(aa(str(X1.value),str(X2.value)))[1]) #X4  Ordenada Calibrada

        #m.Equation([X1>=0])
        #m.Equation([X2>])

        m.Equation([cc(c1P,c1O,X3,X4,minSC)>=0])
        m.Equation([cc(c1P,c1O,X3,X4,maxSC)<=1000])

        m.Equation([cc(c2P,c2O,X3,X4,minSC)>=0])
        m.Equation([cc(c2P,c2O,X3,X4,maxSC)<=1000])

        if(NumSeg>=3):
            m.Equation([cc(c3P,c3O,X3,X4,minSC)>=0])
            m.Equation([cc(c3P,c3O,X3,X4,maxSC)<=1000])
            if(NumSeg==4):
                    m.Equation([cc(c4P,c4O,X3,X4,minSC)>=0])
                    m.Equation([cc(c4P,c4O,X3,X4,maxSC)<=1000])
            else: pass
        else: pass            


        if(NumSeg==2):
            m.Maximize(cc(c1P,c1O,X3,X4,maxSC)+cc(c2P,c2O,X3,X4,maxSC)- cc(c1P,c1O,X3,X4,minSC)- cc(c2P,c2O,X3,X4,minSC) )
        else: pass    


        if(NumSeg==3):
            m.Maximize(cc(c1P,c1O,X3,X4,maxSC)+cc(c2P,c2O,X3,X4,maxSC)+cc(c3P,c3O,X3,X4,maxSC)- cc(c1P,c1O,X3,X4,minSC)- cc(c2P,c2O,X3,X4,minSC) - cc(c3P,c3O,X3,X4,minSC))
        else: pass    

        if(NumSeg==4):
            m.Maximize(cc(c1P,c1O,X3,X4,maxSC)+cc(c2P,c2O,X3,X4,maxSC)+cc(c3P,c3O,X3,X4,maxSC) + cc(c4P,c4O,X3,X4,maxSC) - cc(c1P,c1O,X3,X4,minSC)- cc(c2P,c2O,X3,X4,minSC) - cc(c3P,c3O,X3,X4,minSC) - cc(c4P,c4O,X3,X4,minSC))
        else: pass    


        m.solve(disp=False) 

        m.options.IMODE = 3 # Steady state optimization
        #m.solve() # Solve
        return print('Pendi Calib: ' + str(X3[0])), print('Inter Calib: ' + str(X4[0])), print('Solver: ', m.options.SOLVER)
    else: print("Numero de Segmentos Mal")

        


In [13]:
opti(segmentos)

Pendi Calib: 0.0045612151605
Inter Calib: -0.77235311074
Solver:  3


(None, None, None)